---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

# Libraries

In [328]:
import pandas as pd
import numpy as np

# Import Training Data

In [329]:
train_raw = pd.read_csv('train.csv', encoding = 'ISO-8859-1', index_col = 'ticket_id')
train_raw.head()

/Users/MatiasBerretta/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,MI,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,MI,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,MI,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


# Import Test Data

In [330]:
test_raw = pd.read_csv('test.csv', index_col = 'ticket_id')
test_raw.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,MI,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,MI,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,MI,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


# Address Data

In [331]:
address = pd.read_csv('addresses.csv')

latlon = pd.read_csv('latlons.csv')

address = address.set_index('address').join(latlon.set_index('address'), how = 'left')

address.reset_index(inplace=True)

address.set_index('ticket_id', inplace=True)
address.head()

,address,lat,lon
ticket_id,,,
328722,"-11064 gratiot, Detroit MI",42.406935,-82.995599
350971,"-11871 wilfred, Detroit MI",42.411288,-82.993674
344821,"-15126 harper, Detroit MI",42.406402,-82.957525
24928,"0 10th st, Detroit MI",42.325689,-83.064330
71887,"0 10th st, Detroit MI",42.325689,-83.064330


In [332]:
train_raw_ad = train_raw.join(address)
train_raw_ad.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"2900 tyler, Detroit MI",42.390729,-83.124268
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,"4311 central, Detroit MI",42.326937,-83.135118
22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,MI,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"1449 longfellow, Detroit MI",42.380516,-83.096069
22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,MI,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"1441 longfellow, Detroit MI",42.380570,-83.095919
22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,MI,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"2449 churchill, Detroit MI",42.145257,-83.208233


In [333]:
test_raw_ad = test_raw.join(address)
test_raw_ad.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,address,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,MI,...,20.0,10.0,20.0,0.0,0.0,250.0,NaN,"10041 roseberry, Detroit MI",42.407581,-82.986642
285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,"18520 evergreen, Detroit MI",42.426239,-83.238259
285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,20.0,10.0,10.0,0.0,0.0,140.0,NaN,"18520 evergreen, Detroit MI",42.426239,-83.238259
285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,MI,...,20.0,10.0,20.0,0.0,0.0,250.0,NaN,"1835 central, Detroit MI",42.309661,-83.122426
285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,MI,...,20.0,10.0,10.0,0.0,0.0,140.0,NaN,"1700 central, Detroit MI",42.308830,-83.121116


# Bin Count

In [334]:
bin_count = np.bincount(train_raw_ad['compliance'].dropna().astype(int))
bin_per = (bin_count[0]/bin_count.sum()*100, bin_count[1]/bin_count.sum()*100)

bin_counts_per = {'non-compliant count': bin_count[0], 
                  'non-compliant %': bin_per[0],
                  'compliant count': bin_count[1],
                  'compliant %': 'bin_per[1], %'}

bin_counts_per

{'compliant %': 'bin_per[1], %',
 'compliant count': 11597,
 'non-compliant %': 92.746434826119597,
 'non-compliant count': 148283}

# Clean up train

In [335]:
train_only_features = ['collection_status', 'payment_amount',
                       'payment_status', 'payment_date',
                      'balance_due', 'compliance_detail']

train = train_raw_ad.drop(train_only_features, axis=1)

In [336]:
# Integers
train.dropna(subset = ['compliance'], inplace=True)
train.dropna(subset = ['lat', 'lon'], inplace=True)

# Category use pd.get_dummies
train.loc[:,'compliance'] = train['compliance'].astype(int)
train.loc[:,'agency_name'] = train['agency_name'].astype('category')
train.loc[:,'city'] = train['city'].astype('category')
train.loc[:,'state'] = train['state'].astype('category')
train.loc[:,'zip_code'] = train['zip_code'].astype('category')
train.loc[:,'violation_code'] = train['violation_code'].astype('category')
train.loc[:,'disposition'] = train['disposition'].astype('category')

# Date Time
train.loc[:,'ticket_issued_date'] = pd.to_datetime(train['ticket_issued_date'])
train.loc[:,'hearing_date'] = pd.to_datetime(train['hearing_date'])

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159878 entries, 22056 to 284333
Data columns (total 30 columns):
agency_name                   159878 non-null category
inspector_name                159878 non-null object
violator_name                 159852 non-null object
violation_street_number       159878 non-null float64
violation_street_name         159878 non-null object
violation_zip_code            0 non-null float64
mailing_address_str_number    157320 non-null float64
mailing_address_str_name      159875 non-null object
city                          159878 non-null category
state                         159794 non-null category
zip_code                      159877 non-null category
non_us_str_code               3 non-null object
country                       159878 non-null object
ticket_issued_date            159878 non-null datetime64[ns]
hearing_date                  159651 non-null datetime64[ns]
violation_code                159878 non-null category
violation_descript

# Columns to keep

In [337]:
#train = train[train['city'] == 'Detroit']

train_to_keep = ['fine_amount', 'admin_fee', 'discount_amount','clean_up_cost',
                 'state_fee', 'late_fee','judgment_amount', 'lat','lon']

# Encode Functions

In [338]:
def hot_encode(df, dummy_col):
    '''returns new dataframe with encoded columns for selected variable'''
    hot = pd.get_dummies(dummy_col)
    df = pd.merge(df, hot, right_index=True, left_index=True)
    
    return df 

# Encoding disposition

### Train

In [339]:
train_reduced = train.loc[:,train_to_keep]
X_train = hot_encode(train_reduced, train['disposition'])
X_train.shape

(159878, 13)

### Test

In [358]:
X_test = test_raw_ad.loc[:, train_to_keep]
#X_test.dropna(subset = ['lat', 'lon'], inplace=True)

X_test = hot_encode(X_test, test_raw['disposition'])

X_test['lat'].fillna(method = 'pad', inplace=True)
X_test['lon'].fillna(method = 'pad', inplace=True)
#for i in zip(X_train.columns, X_test.columns):
#    print(list(i))


remove_from_test = ['Responsible (Fine Waived) by Admis',
                   'Responsible - Compl/Adj by Default',
                   'Responsible - Compl/Adj by Determi',
                   'Responsible by Dismissal']

X_test.drop(remove_from_test, axis=1, inplace=True)

X_test.dropna(subset = ['lat', 'lon'], inplace=True)

X_test.shape

(61001, 13)

### Encoding violation code

In [298]:
#X_train = hot_encode(X_train, train['violation_code'])
#X_train.info()

#X_test = hot_encode(X_test, test_raw['violation_code'])
#X_test.shape

### Encoding state

In [299]:
#X_train = hot_encode(X_train, train['state'])
#X_train.info()

#X_test = hot_encode(X_test, test_raw['state'])
#X_test.shape

# Split data

In [348]:
y_train = train.loc[:,'compliance']

(X_train.shape, y_train.shape)
(X_train.shape[1], X_test.shape[1])

(13, 13)

# Random Forest 

In [359]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rforest = RandomForestClassifier(n_estimators=100, random_state = 0)

rforest.fit(X_train, y_train)

y_score = rforest.predict(X_train)

roc_auc_score(y_train, y_score)

y_pred = rforest.predict_proba(X_test)[:,1]

print(y_pred)

[ 0.04  0.    0.   ...,  0.01  0.    0.79]


# Logistic Regression - Sanity Check

In [326]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import roc_auc_score

#lreg = LogisticRegression()

#lreg.fit(X_train, y_train)
#y_score = lreg.predict(X_train)

#roc_auc_score(y_train, y_score)

0.55168382083150491

# Blight model

Must be of length 61001

In [361]:
import pandas as pd
import numpy as np

def blight_model():
    
    X_test['compliance'] = y_pred
    
    result = X_test['compliance']
    
    return result

blight_model()

ticket_id
284932    0.040000
285362    0.000000
285361    0.000000
285338    0.010000
285346    0.000000
285345    0.170000
285347    0.010000
285342    0.910000
285530    0.026667
284989    0.000000
285344    0.000000
285343    0.000000
285340    0.000000
285341    0.316333
285349    0.040000
285348    0.060000
284991    0.000000
285532    0.000000
285406    0.000000
285001    0.000000
285006    0.000000
285405    0.000000
285337    0.000000
285496    0.000000
285497    0.020000
285378    0.000000
285589    0.000000
285585    0.000000
285501    0.103333
285581    0.200000
            ...   
376367    0.000000
376366    0.110000
376362    0.050000
376363    0.550000
376365    0.000000
376364    0.110000
376228    0.797692
376265    0.002500
376286    0.310000
376320    0.020000
376314    0.000000
376327    0.730000
376385    0.120000
376435    0.900000
376370    1.000000
376434    0.000000
376459    0.000000
376478    0.000000
376473    0.000000
376484    0.040000
376482    0.000000
37